<a href="https://colab.research.google.com/github/Naheli98/AML/blob/main/AML%20Assignment%205/%20AML_Assignment_5_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"naheli","key":"878270e6d0586daea529f8a3658358b5"}'}

In [2]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [3]:
!kaggle datasets download -d abhi8923shriv/sentiment-analysis-dataset

 97% 53.0M/54.4M [00:04<00:00, 16.2MB/s]
100% 54.4M/54.4M [00:04<00:00, 12.0MB/s]


In [4]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.7 MB/s eta 0:00:00


In [6]:
!unzip /content/sentiment-analysis-dataset.zip -d /content/sentiment-analysis-dataset/

Archive:  /content/sentiment-analysis-dataset.zip
  inflating: /content/sentiment-analysis-dataset/test.csv  
  inflating: /content/sentiment-analysis-dataset/testdata.manual.2009.06.14.csv  
  inflating: /content/sentiment-analysis-dataset/train.csv  
  inflating: /content/sentiment-analysis-dataset/training.1600000.processed.noemoticon.csv  


In [7]:
import os, sys, random
import datetime, time, copy
import torch, warnings

import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import transformers

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler
from torch.optim import AdamW
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, f1_score
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import get_linear_schedule_with_warmup
from PIL import Image

warnings.filterwarnings("ignore")
%matplotlib inline

In [8]:
use_gpu = torch.cuda.is_available()
print('Using GPU: ', use_gpu)

if use_gpu == True:
	device = torch.device("cuda")
else:
	device = torch.device("cpu")

Using GPU:  True


In [11]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [9]:

df = pd.read_csv("/content/sentiment-analysis-dataset/train.csv", encoding= 'unicode_escape')
df.head()


,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [10]:
df = df.dropna()
df = df[["selected_text","sentiment"]]

In [11]:
df['sentiment'] = df['sentiment'].map({'neutral': 1, 'negative': 0, 'positive': 2})

In [12]:
df['sentiment'].value_counts(normalize = True)

1    0.404549
2    0.312300
0    0.283151
Name: sentiment, dtype: float64

In [13]:

train_text, temp_text, train_labels, temp_labels = train_test_split(df['selected_text'], df['sentiment'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=df['sentiment'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)
     

In [14]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased',return_dict=False)

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased',return_dict=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
max_seq_len = 25

In [16]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

In [17]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [18]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [19]:
for param in bert.parameters():
    param.requires_grad = False
     

In [20]:

class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert       
      self.dropout = nn.Dropout(0.1)      
      self.relu =  nn.ReLU()
      self.fc1 = nn.Linear(768,512)      
      self.fc2 = nn.Linear(512,3)
      self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, sent_id, mask):
      _, cls_hs = self.bert(sent_id, attention_mask=mask)      
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc2(x)      
      x = self.softmax(x)
      return x

In [21]:
model = BERT_Arch(bert)
model = model.to(device)
     

In [22]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr = 1e-3)

In [23]:

from sklearn.utils.class_weight import compute_class_weight

class_wts = compute_class_weight(class_weight = 'balanced',classes = np.unique(train_labels),y= train_labels)

print(class_wts)
     

[1.17716174 0.82395271 1.06742134]


In [24]:

weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

cross_entropy  = nn.NLLLoss(weight=weights) 

epochs = 10

In [25]:

print(weights)

tensor([1.1772, 0.8240, 1.0674], device='cuda:0')


In [26]:

def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0

  total_preds=[]

  for step,batch in enumerate(train_dataloader):

    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    model.zero_grad()        

    preds = model(sent_id, mask)

    loss = cross_entropy(preds, labels)

    total_loss = total_loss + loss.item()

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()

    preds=preds.detach().cpu().numpy()

    total_preds.append(preds)

  avg_loss = total_loss / len(train_dataloader)
  
 
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [27]:


def evaluate():
  
  print("\nEvaluating...")
  
  
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  
  total_preds = []

  
  for step,batch in enumerate(val_dataloader):
    
    
    if step % 50 == 0 and not step == 0:
      
      
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      

      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  
  avg_loss = total_loss / len(val_dataloader) 

  
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [28]:

best_valid_loss = float('inf')


train_losses=[]
valid_losses=[]


for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10
  Batch    50  of    602.
  Batch   100  of    602.
  Batch   150  of    602.
  Batch   200  of    602.
  Batch   250  of    602.
  Batch   300  of    602.
  Batch   350  of    602.
  Batch   400  of    602.
  Batch   450  of    602.
  Batch   500  of    602.
  Batch   550  of    602.
  Batch   600  of    602.

Evaluating...
  Batch    50  of    129.
  Batch   100  of    129.

Training Loss: 0.820
Validation Loss: 0.647

 Epoch 2 / 10
  Batch    50  of    602.
  Batch   100  of    602.
  Batch   150  of    602.
  Batch   200  of    602.
  Batch   250  of    602.
  Batch   300  of    602.
  Batch   350  of    602.
  Batch   400  of    602.
  Batch   450  of    602.
  Batch   500  of    602.
  Batch   550  of    602.
  Batch   600  of    602.

Evaluating...
  Batch    50  of    129.
  Batch   100  of    129.

Training Loss: 0.694
Validation Loss: 0.630

 Epoch 3 / 10
  Batch    50  of    602.
  Batch   100  of    602.
  Batch   150  of    602.
  Batch   200  of    602.
  B

In [29]:
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [30]:
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()
     

In [31]:
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.82      0.69      0.75      1167
           1       0.82      0.81      0.81      1668
           2       0.75      0.86      0.80      1287

    accuracy                           0.79      4122
   macro avg       0.80      0.79      0.79      4122
weighted avg       0.80      0.79      0.79      4122



In [32]:

pd.crosstab(test_y, preds)

col_0,0,1,2
row_0,,,
0,808,178,181
1,126,1347,195
2,49,126,1112
